# Numerical methods for solving nonlinear equations 

Let us consider a simple chemical system: **0.1M H₂SO₄** in water. What is its pH? For simplicity, we will assume activity coefficients are one, the activity of water is one, and we will set standard concentrations to one. Thus, we can simply write the concentration as $a(x) = [x] $, where square brackets denote concentration.

1) **First dissociation step**:
   $$
   \text{H₂SO₄} + \text{H₂O} \rightleftharpoons \text{H₃O}^+ + \text{HSO₄}^- 
   $$
   with equilibrium constant:
   $$
   K_1 = \frac{[ \text{H₃O}^+ ][ \text{HSO₄}^- ]}{[ \text{H₂SO₄} ]} = 1000
   $$

2) **Second dissociation step**:
   $$
   \text{HSO₄}^- + \text{H₂O} \rightleftharpoons \text{H₃O}^+ + \text{SO₄}^{2-}
   $$
   with equilibrium constant:
   $$
   K_2 = \frac{[ \text{H₃O}^+ ][ \text{SO₄}^{2-} ]}{[ \text{HSO₄}^- ]} = 0.012
   $$

   
Then there is of course the infamous water dissociation:

$$
2\text{H₂O} \rightleftharpoons \text{H₃O}^+ + \text{OH}^-
$$
Let us be real chemists and ignore that.

Starting with mass balance:
$$
2\text{H₂O} \rightleftharpoons \text{H₃O}^+ + \text{OH}^-
$$
and electroneutrality:
$$
[ \text{HSO₄}^- ] + 2[ \text{SO₄}^{2-} ] = [ \text{H₃O}^+ ]
$$

Now, we combine the equilibrium expressions $ K_1$ and $ K_2 $ with the mass balance and electroneutrality conditions to obtain a cubic equation. The derivation is straightforward and leads us to:

$$
\frac{c}{[ \text{H₃O}^+ ]} = \frac{1 + \frac{K_1}{[ \text{H₃O}^+ ]} + \frac{K_1 K_2}{[ \text{H₃O}^+ ]^2}}{\frac{K_1}{[ \text{H₃O}^+ ]} + 2 \frac{K_1 K_2}{[ \text{H₃O}^+ ]^2}}
$$
To simplify this beauty, introduce $x = [ \text{H₃O}^+ ]$:
$$
x^3 + K_1 x^2 + (K_1 K_2 - c K_1) x - 2 c K_1 K_2 = 0
$$

and lets not forget:
$$
\text{pH} = -\log_{10} x
$$

To determine the pH, we need to find the root $ x $ of this cubic equation. <br></br>
Before diving into the how let us consider some logical values for x.
We know concentration cannot be less than 0 and we know it cannot be more than 0.2M. Thus, we are looking for a root of cubic equation on the interval $[0, 0.2]$

# Methods for solving nonlinear equations
1) Bisection
2) Secant method
3) Tangent method

## Bisection

<img src="bisection.png" alt="Cubic spline" width="500" height="250"/>

Given $f(x)$ we can use bisection to find its root on interval $[a,b]$ if:
- $f(x)$ is continuous
- If there is and odd zero of $f(x)$ on interval $[a,b]$, that is $f(a)f(b)<0$

1) **Pick a midpoint** $c = \frac{a + b}{2} $
2) **Check if** $f(a) \cdot f(c) < 0 $
   - If it is, the root is between $ a $ and $ c $. Set $ b = c $ and repeat the process.
   - If it is not, the root is between $ c $ and $b $. Set $ a = c $ and repeat the process.

Naturally, one wonders: **How long should the procedure run?**
We employ one of two convergence criteria:

- $ \left| f(c_n) \right| < \epsilon $
- $ \left| b_n - a_n \right| < \epsilon $

Computationally, it is more reasonable to use the second condition, which is based on the interval length.
We can estimate the number of steps required as follows:

1) In the first step: 
   $$
   \frac{b_1 - a_1}{2} = \left| c_1 - c \right| \quad \text{(where \( c \) is the root)}
   $$

2) In the second step: 
   $$
   \left| c_2 - c \right| = \frac{b_2 - a_2}{2} = \frac{b_1 - a_1}{4}
   $$

3) In the third step: 
   $$
   \left| c_3 - c \right| = \frac{b_3 - a_3}{2} = \frac{b_1 - a_1}{8}
   $$

   And so on.

In general, after $n $ steps:
$$
\left| c_n - c \right| = \frac{b_n - a_n}{2} = \frac{b_1 - a_1}{2^n}
$$

To stop when the error is less than a tolerance $ \epsilon$, we need:
$$
\frac{b_1 - a_1}{2^n} < \epsilon
$$

Taking the logarithm:
$$
n > \frac{\ln(b_1 - a_1) - \ln(\epsilon)}{\ln(2)}
$$
Thus, the number of iterations required to achieve the desired precision is proportional to the logarithm of the initial interval length divided by the tolerance.

In [45]:
def bisection(f,a,b,ep=1e-6):
    if (f(a)*f(b)>0): return 
    while abs(a-b)>ep:
        c=(a+b)/2
        if (f(a)*f(c)<0):
            b=c
        else:
            a=c
    return c

## Secant method

<img src="secant.png" alt="Cubic spline" width="500" height="250"/>

Given $f(x)$ we can use bisection to find its root on interval $[a,b]$ if:
- $f(x)$ is continuous
- If there is and odd zero of $f(x)$ on interval $[a,b]$, that is $f(a)f(b)<0$

The algorithm  is essentially the same as bisection. It first determines a midpoint, and then checks where to shrink the interval.
The secant method determines a midpoint by drawing a line between two points $ (a, f(a)) $ and $ (b, f(b)) $. The slope of this line is:

$$
k = \frac{f(b) - f(a)}{b - a} = \frac{y - f(a)}{x - a}
$$
Rearranging this equation to solve for $ y $:

$$
y = f(a) + \frac{f(b) - f(a)}{b - a} (x - a)
$$

We choose the new point where this line crosses the x-axis, which is at:

$$
x = a - \frac{f(a) \cdot (b - a)}{f(b) - f(a)}
$$

Now, we set $ x = c $ as the "midpoint."

The outline of the algorithm is the same as for bisection:
1) **Pick a midpoint** $c = a - \frac{f(a) \cdot (b - a)}{f(b) - f(a)}  $
2) **Check if** $f(a) \cdot f(c) < 0 $
   - If it is, the root is between $ a $ and $ c $. Set $ b = c $ and repeat the process.
   - If it is not, the root is between $ c $ and $b $. Set $ a = c $ and repeat the process.


We again employ one of two convergence criteria:

- $ \left| f(c_n) \right| < \epsilon $
- $ \left| c_{n} - c_{n-1} \right| < \epsilon $

In this case, it is more practical to implement the first criterion, which focuses on the function value at the midpoint.

For this method, it is difficult to determine the exact number of steps required. However, it is generally the case that fewer steps are needed compared to the bisection method. This is because the secant method often converges faster, especially when the initial guesses are close to the root.

In [88]:
def secant(f,a,b,ep=1e-6):
    if (f(a)*f(b)>0): return 
    c=a #random guess for start
    while abs(f(c))>ep:
        c=a - f(a)*(b-a)/(f(b)-f(a))
        if (f(a)*f(c)<0):
            b=c
        else:
            a=c
    return c

## Tangent method (newton-Raphson)

<img src="tangent.png" alt="Cubic spline" width="500" height="250"/>

Given $f(x)$ and a random guess $x_0$ we can find a root of $f(x)$ provided:
- $f(x)$ is continuously differentiable (at least around zero)
- $x_0$ is a sufficient guess

#### Idea
We start with a point $ (a_0, f(a_0)) $. From the derivative of $ f(x)$ at $a_0 $, we can construct the tangent line:
$$
y = f(a_0) + (x - a_0) f'(a_0)
$$
This tangent intersects the x-axis when $ y = 0$, which gives us the equation:
$$
x = a_0 - \frac{f(a_0)}{f'(a_0)}
$$

This gives us a new point $ a_1 = x $ and $ f(a_1) $, which should be closer to the actual root.


#### Algorithm

Based on the above outline, we can construct the following algorithm:

1) Compute $ f(a) $ and $ f'(a)$.
2) Update $a $ by: 
   $
   a_{\text{new}} = a - \frac{f(a)}{f'(a)}
   $
3) Repeat the process.

 
We can choose from the following three convergence criteria:

- $ \left| a_n - a_{n-1} \right| < \epsilon $
- $ \left| f(a_n) \right| < \epsilon $
- $ \left| \frac{f'(a)}{f(a)} \right| < \epsilon $

Most practial to use is the third criterion.


In [31]:
def tangent(f, x0, ep=1e-6, max_steps=1000, df=None): #Here I will make a failsafe in case it doesnt converge
    df=df if df!=None else lambda x: (f(x+h)-f(x-h))/(2*h)
    for step in range(max_steps):
        difference=f(x0)/df(x0)
        if abs(difference)<ep: return x0
        x0-=difference
    return 
    

## Calculation of pH

Now after a long detour, we can finally return to our calculation of pH. We are trying to find the root of:
$$
x^3 + K_1 x^2 + (K_1 K_2 - c K_1) x - 2 c K_1 K_2 = 0
$$

In [90]:
from math import log10
K1=1000
K2=0.012
c=0.1
a=0
b=0.2

def concentration_equation(x):
    return x*x*x + K1*x*x + (K1*K2 - c*K1)*x - 2*K1*K2*c # here I am definign a function only for clarity, it is much better to use lambda
def derivative_equation(x):
    return 3*x*x + 2*x*K1 + (K1*K2 - K1*c) #again much better to just use numerical in reality

x_b=bisection(concentration_equation,a,b)
x_s=secant(concentration_equation,a,b)
x_t=tangent(concentration_equation,(a+b)/2,df=derivative_equation)

print(f"pH with bisection is {-log10(x_b)}")
print(f"pH with secant is {-log10(x_s)}")
print(f"pH with tangent is {-log10(x_t)}")

pH with bisection is 0.9592469714169428
pH with secant is 0.9592464202531963
pH with tangent is 0.9592464031907413
